In [1]:
import pandas as pd
import numpy as np
import csv
import json
from scipy.stats import linregress
from statsmodels.stats.weightstats import ztest as ztest

# local code
from source.interactive_plots import interactive_linear_regression_calibration_plot
from source.get_elements      import get_elements
from source.outliers          import dixon_test

Load data from XRF analyses

In [2]:
xrf_data = pd.read_csv("../data/interim/xrf_data_clean.csv") # load xrf data
# drop uncertainty columns 
xrf_data.drop([column for column in xrf_data.columns if column.endswith("+/-")], axis=1, inplace=True)

# Calibration

Load data for standard reference materials and clean

In [3]:
srm_data = pd.read_csv("../data/interim/standard_reference_material_certified_values.csv") # load SRM data
srm_data.drop(["1SD", "95% Confidence Low", "95% Confidence High"], axis=1, inplace=True) # drop unnecessary columns

# clean SRM data
for row in srm_data.iterrows(): 
    row[1]["Sample ID"] = row[1]["Sample ID"].lower()
    row[1]["Analyte"] = row[1]["Analyte"].split(",",1)[0]

    # clean certified value
    cert_val = row[1]["Certified Value"]
    if cert_val.startswith("<"): 
        row[1]["Certified Value"] = float(cert_val.lstrip("< ")) / 2 # replace BDL with half value
    else:
        row[1]["Certified Value"] = float(row[1]["Certified Value"])

    # clean units
    row[1]["Units"] = row[1]["Units"].lstrip("(").rstrip(")")
    if row[1]["Units"] == "wt.%":  # convert units
       row[1]["Certified Value"] = row[1]["Certified Value"] * 1e4
       row[1]["Units"] = "ppm"

## Remove standard data for a given element that is unsuitable for calibration

Compare the measured concentration of each element for a given standard to the distribution of measured concentrations for that element across all non-standards. If the measured concentration 

    1.) falls outside the range of the distribution 
    2.) is more than three standard deviations from the mean 

that value is set to `NaN` (all values set to `NaN` are filtered out during calibration). 

Note: I should consider conducting a test to determine if each distribution is normally or log-normally distributed; if the latter, I should log-transform the data and then calculate the z-score. 

In [4]:
def z_score(data, value): 
    mean = np.mean(data)
    std = np.std(data)

    return (value - mean) / std

In [5]:
non_standards_data = xrf_data.loc[xrf_data["qaqc_type"]!="standard"]

outlier_stddev_cutoff = 5

for standard in xrf_data.loc[xrf_data["qaqc_type"]=="standard", "sample_id"].unique(): 
    for date in xrf_data.loc[xrf_data["sample_id"]==standard, "date"].unique():
        for element in get_elements(xrf_data.columns.to_list()): 
            standard_date_element = xrf_data.loc[(xrf_data["sample_id"]==standard) & (xrf_data["date"]==date)][element].values

            if (non_standards_data[element] > standard_date_element[0]).all() | \
               (non_standards_data[element] < standard_date_element[0]).all(): 
                
                # print(non_standards_data[element].to_numpy())
                z = z_score(non_standards_data[element], standard_date_element[0])
                if abs(z) > outlier_stddev_cutoff: 
                    # print(standard + ", " + date + ", " + element + ", " + str(standard_date_element[0]) + ", " + str(z))

                    xrf_data.loc[(xrf_data["sample_id"]==standard) & (xrf_data["date"]==date), element] = np.nan

In [6]:
xrf_data[xrf_data["sample_id"]=="TE2-008"]["Mo"]

38    2.0
Name: Mo, dtype: float64

Get the elements analyzed by the XRF and for which concentrations are reported for one or more standard reference materials

In [7]:
elements = get_elements(
                        list(
                            set(srm_data["Analyte"].unique()) & \
                            set(xrf_data.columns.to_list())
                            )
                        )

Construct a linear regression model for each element in order to predict the true concentration from the measured concentration

In [8]:
# initialize dictionary to hold lin. reg. models for each element
reg = {}
for element in elements: 

    # get IDs of standard reference materials
    srm = srm_data.loc[srm_data["Analyte"]==element]["Sample ID"].unique() 

    ## TRAIN 
    # limit training data to standards for which we have standard reference material info for the element at hand
    data_train = xrf_data.loc[(xrf_data["qaqc_type"]=="standard") & (xrf_data["sample_id"].isin(srm))]
    data_train = data_train.dropna(subset=[element]) # change to true condition statement

    x_train = [srm_data.loc[
                            (srm_data["Sample ID"]==sample) & \
                            (srm_data["Analyte"]==element)
                            ]["Certified Value"].values[0] for sample in data_train["sample_id"]]
    x_train = np.array(x_train)
    y_train = data_train[element].to_numpy()

    model = linregress(x_train, y_train) # fit linear regression model

    if model.slope != 0: #only use calibration curve if meaningful (i.e., if variance in dep. var. explained by variance in indep. var.)
        # invert model so that measured concentration is independent var. and true concentration is dependent var. (i.e., y = m*x + b --> x = (1/m)*y - (b/m))
        intercept_inv = -model.intercept / model.slope
        slope_inv = model.slope ** -1

        ## PREDICT (i.e., calibrate)
        data_predict = xrf_data.loc[xrf_data["qaqc_type"]!="standard"]
        data_predict = data_predict.dropna(subset=[element])

        x_predict = data_predict[element]
        x_predict = x_predict.to_numpy()
        y_predict = slope_inv * x_predict + intercept_inv

        ## Save model results
        reg[element] = {} # initialize empty dict to save model results

        reg[element]["model"]                 = model
        reg[element]["x_train"]               = x_train
        reg[element]["y_train"]               = y_train
        reg[element]["score"]                 = model.rvalue
        reg[element]["y-intercept std error"] = model.intercept_stderr
        reg[element]["slope_inv"]             = slope_inv
        reg[element]["intercept_inv"]         = intercept_inv
        reg[element]["x_predict"]             = x_predict
        reg[element]["y_predict"]             = y_predict
        
        xrf_data.loc[xrf_data["qaqc_type"]!="standard", element] = y_predict

/Users/robertcollar/miniconda3/envs/EPSC-552-Mont-St-Hilaire/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/Users/robertcollar/miniconda3/envs/EPSC-552-Mont-St-Hilaire/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Display the results of the calibration

In [9]:
xrf_data[xrf_data["sample_id"]=="TE2-008"]["Mo"]

38    1.418657
Name: Mo, dtype: float64

In [10]:
dropdown_buttons = {
    "data": 
        {
            "name": "Elements", 
            "columns": list(reg.keys())
        }
    }

interactive_linear_regression_calibration_plot(dropdown_buttons, reg, x_axis_label="True concentration (ppm)", y_axis_label="Measured concentration (ppm)", title="")

Calculate the detection limit for each element. 

In [11]:
for element in elements: 
    detection_limit = reg[element]["slope_inv"] * (reg[element]["model"].intercept + 3*reg[element]["y-intercept std error"]) + reg[element]["intercept_inv"]
    reg[element]["detection_limit"] = detection_limit

Apply detection limit to dataset.

In [12]:
for element in reg.keys(): 
    limit = reg[element]["detection_limit"]
    xrf_data[element].where(xrf_data[element] >= limit, other=f"<{limit}", inplace=True)

In [13]:
xrf_data[xrf_data["sample_id"]=="TE2-008"]["Mo"]

38    <2.4797694423678664
Name: Mo, dtype: object

# Evaluating Precision

In [14]:
lab_duplicates = xrf_data[xrf_data["qaqc_type"]=="lab duplicate"]

lab_parent_indexes = []

for row in lab_duplicates.iterrows(): 
    lab_dup_id = row[1]["sample_id"]
    lab_parent_id = lab_dup_id.rstrip('L')

    condition = xrf_data["sample_id"] == lab_parent_id
    index = xrf_data.index[condition][0]
    lab_parent_indexes.append(index)

lab_parents = xrf_data.iloc[lab_parent_indexes]

lab_pairs = pd.concat([lab_duplicates, lab_parents])

In [15]:
def half_detection_limit(x): 
    if isinstance(x, str): 
        if x.startswith("<"): 
            x = float(x.lstrip("< ")) / 2 # replace BDL with half value
    return x

In [16]:
xrf_data[xrf_data["sample_id"]=="TE2-008"]["Mo"]

38    <2.4797694423678664
Name: Mo, dtype: object

In [17]:
lab_pairs[lab_pairs["sample_id"]=="TE2-008L"]["Mo"]

48    <2.4797694423678664
Name: Mo, dtype: object

In [18]:
lab_parents[lab_parents["sample_id"]=="TE2-008"]["Mo"]

38    <2.4797694423678664
Name: Mo, dtype: object

In [19]:
lab_pair_diffs = lab_parents.copy()
lab_duplicates = lab_duplicates.copy()

elements_dup = get_elements(xrf_data.columns.to_list())

for row in lab_pair_diffs.iterrows():
    parent_id = row[1]["sample_id"]
    duplicate_id = parent_id + "L"

    parent_entry = row[1].to_frame().transpose()
    parent_entry.reset_index(inplace=True)
    parent_values = parent_entry.loc[:, elements_dup]

    duplicate_entry = lab_duplicates.loc[lab_duplicates["sample_id"]==duplicate_id]
    duplicate_entry.reset_index(inplace=True)
    duplicate_values = duplicate_entry.loc[:, elements_dup]
    
    condition = lab_pair_diffs["sample_id"] == parent_id
    lab_pair_diffs.loc[condition, elements_dup] = parent_values.apply(pd.to_numeric, errors="coerce").\
                                                  subtract(duplicate_values.apply(pd.to_numeric, errors="coerce")).values

    parent_values = parent_values.applymap(half_detection_limit)
    duplicate_values = duplicate_values.applymap(half_detection_limit)

    pair_values = pd.concat((parent_values, duplicate_values))

    condition = xrf_data["sample_id"] == parent_id
    xrf_data.loc[condition, elements_dup] = pair_values.mean().values
    
    condition = xrf_data["sample_id"] == duplicate_id
    xrf_data.drop(xrf_data[condition].index, inplace=True)
xrf_data.reset_index(inplace=True)

lab_pair_diffs = lab_pair_diffs[["sample_id"] + elements_dup]

In [20]:
lab_pair_diffs[lab_pair_diffs["sample_id"]=="TE2-008"]["Mo"]

38    NaN
Name: Mo, dtype: object

In [21]:
def std_dev_from_pairs(diffs): 
    diffs = np.asarray(diffs)
    n = diffs.size
    
    std_dev = np.sqrt(np.sum(np.square(diffs)) / (2 * n))

    return std_dev, n

In [22]:
def rel_std_dev(element, data, std_dev): 
    data = data.applymap(half_detection_limit)
    mean = data.loc[xrf_data["qaqc_type"] != "standard", element].mean()
    rsd = 100 * std_dev / mean

    return rsd, mean

In [23]:
lab_pair_diffs[element]

19             NaN
30       66.114723
38     1168.026767
74      330.573613
72     1079.873804
62             NaN
57      396.688336
67     3327.774375
102     374.650095
97     2380.130016
92      352.611854
87    -1454.523899
107     462.803059
120   -2336.053534
123    -132.229445
128            NaN
140   -1652.868067
145            NaN
150    -352.611854
153    -374.650095
190            NaN
139    -154.267686
180     -22.038241
175    -617.070745
170    -550.956022
204     264.458891
219    -374.650095
Name: Zr, dtype: object

In [24]:
analytical_precision = {}
xrf_data_bdl_replaced = xrf_data.applymap(half_detection_limit)
for element in elements_dup: 
    diffs = lab_pair_diffs[element]

    if False in list(diffs.isnull()): # ensure that there are >= 1 valid data values
        diffs.dropna(inplace=True)

        if (len(diffs) >=3): # min. for Dixon's Q Test
            diffs = dixon_test(list(diffs), left=True, right=True, confidence_level=95)[0]
        std_dev, n = std_dev_from_pairs(diffs)
        
        mean = xrf_data_bdl_replaced.loc[xrf_data_bdl_replaced["qaqc_type"] != "standard", element].mean()
        rsd = 100 * std_dev / mean

        analytical_precision[element] = {}
        analytical_precision[element]["standard deviation"] = std_dev
        analytical_precision[element]["number of pairs"] = n
        analytical_precision[element]["rsd"] = rsd
        analytical_precision[element]["mean"] = mean

In [25]:
with open("../data/interim/analytical_precision.json", "w") as outfile: 
    json.dump(analytical_precision, outfile, indent=4)

## Field heterogeneity

Get field duplicate-parent pairs

In [26]:
field_duplicates = xrf_data[xrf_data["qaqc_type"]=="field duplicate"]
field_parent_indexes = []

for row in field_duplicates.iterrows(): 
        field_dup_id = row[1]["sample_id"]
        field_parent_id = field_dup_id.rstrip('F')

        condition = xrf_data["sample_id"] == field_parent_id
        index = xrf_data.index[condition][0]
        field_parent_indexes.append(index)

field_parents = xrf_data.iloc[field_parent_indexes]

field_pairs = pd.concat([field_duplicates, field_parents])

Calculate difference between each duplicate-parent pair

In [27]:
elements_dup = get_elements(xrf_data.columns.to_list())

field_pair_diffs = field_parents.copy()
field_pair_diffs.loc[:, elements_dup] = field_pair_diffs.loc[:, elements_dup].apply(pd.to_numeric, errors="coerce")

field_duplicates = field_duplicates.copy()
field_duplicates.loc[:, elements_dup] = field_duplicates.loc[:, elements_dup].apply(pd.to_numeric, errors="coerce")

for row in field_pair_diffs.iterrows():
    parent_id = row[1]["sample_id"]
    duplicate_id = parent_id + "F"

    parent_entry = row[1].to_frame().transpose()
    parent_entry.reset_index(inplace=True)
    parent_values = parent_entry.loc[:, elements_dup]

    duplicate_entry = field_duplicates.loc[field_duplicates["sample_id"]==duplicate_id]
    duplicate_entry.reset_index(inplace=True)
    duplicate_values = duplicate_entry.loc[:, elements_dup]
    
    condition = field_pair_diffs["sample_id"] == parent_id
    field_pair_diffs.loc[condition, elements_dup] = parent_values.subtract(duplicate_values).values


field_pair_diffs = field_pair_diffs[["sample_id"] + elements_dup]

Calculate field heterogeneity

In [39]:
field_heterogeneity = {}
for element in elements_dup: 
    diffs = field_pair_diffs[element]
    
    if False in list(diffs.isnull()): # ensure that there are >= 1 valid data values
        diffs.dropna(inplace=True) 

        if len(diffs) >=3: # min. for Dixon's Q Test 
            diffs = dixon_test(list(diffs), left=True, right=True, confidence_level=95)[0]
        std_dev, n = std_dev_from_pairs(diffs)
        mean = xrf_data_bdl_replaced.loc[xrf_data_bdl_replaced["qaqc_type"] != "standard", element].mean()


        if element in analytical_precision.keys(): # analytical precision must be calculated to determine field heterogeneity
            field_heterogeneity[element] = {}
            field_heterogeneity[element]["heterogeneity"] = std_dev - analytical_precision[element]["standard deviation"]
            field_heterogeneity[element]["relative heterogeneity"] = 100 * field_heterogeneity[element]["heterogeneity"] / mean
            field_heterogeneity[element]["number of pairs"] = n

Save field heterogeneity data

In [41]:
with open("../data/interim/field_heterogeneity.json", "w") as outfile: 
    json.dump(field_heterogeneity, outfile, indent=4)

In [42]:
# alpha= 0.05
# element = "Ba"
# normality_test_data = xrf_data.loc[xrf_data["qaqc_type"]!="standard", element]
# k, p = stats.normaltest(normality_test_data)
# if p < alpha:
#     print("reject null hypothesis that sample is normall distributed")

In [43]:
# export data to csv file
xrf_data.to_csv('../data/interim/xrf_data_calib.csv')

In [44]:

    # xrf_data.to_excel('../data/interim/xrf_data_calib.xlsx', na_rep="NaN")

In [45]:
dataset_evaluation = dict.fromkeys(get_elements(xrf_data.columns))
for element in dataset_evaluation.keys(): 
    dataset_evaluation[element] = {"calibration":          {}, \
                                   "analytical precision": {}, \
                                   "field heterogeneity":  {}}

for element in reg.keys(): 
    dataset_evaluation[element]["calibration"] = {"R-squared": round(reg[element]["score"], 2)}

for element in analytical_precision.keys(): 
    dataset_evaluation[element]["analytical precision"] = {"standard deviation": round(analytical_precision[element]["standard deviation"], 2), \
                                                           "rsd":                round(analytical_precision[element]["rsd"],                2), \
                                                           "number of pairs":    round(analytical_precision[element]["number of pairs"],    2)}

for element in field_heterogeneity.keys(): 
    dataset_evaluation[element]["field heterogeneity"] = {"heterogeneity":   round(field_heterogeneity[element]["heterogeneity"],   2), \
                                                          "number of pairs": round(field_heterogeneity[element]["number of pairs"], 2)}   

In [46]:
discard = {}
usable_dataset_evaluation = dataset_evaluation.copy()
for element in dataset_evaluation.keys(): 
    if dataset_evaluation[element]["calibration"] == {}:
        discard[element] = "srm values missing"

    elif dataset_evaluation[element]["calibration"]["R-squared"] <= 0.3:
        discard[element] = "poor calibration"

    elif dataset_evaluation[element]["analytical precision"] == {}: 
        discard[element] = "insufficient data to calculate analytical precision"
        
    elif dataset_evaluation[element]["analytical precision"]["number of pairs"] < 9:
        discard[element] = "insufficient data to calculate analytical precision"
    
    elif dataset_evaluation[element]["analytical precision"]["rsd"] > 30:
        discard[element] = "RSD is too high"

for element in discard.keys():
    usable_dataset_evaluation.pop(element)

Clean analysis-ready dataset

In [47]:
xrf_data = xrf_data.applymap(half_detection_limit)